### Utility notebook for stats calculation, prompts sampling and post-processing for GPT-3 generated dataset

In [1]:
import pandas as pd
import csv

In [19]:
# train stats
train_file = open('eca-train-cleaned.tsv')
train_tsv = csv.reader(train_file, delimiter="\t")
total_length = 0
total = 0
for i, line in enumerate(train_tsv):
    if i == 0: continue
    total += 1
    total_length += len(line[0].split())
    # print(len(line[0].split()))

print(total)
print(total_length)
print(total_length / total)

3012
187611
62.287848605577686


In [20]:
# dev stats
dev_file = open('eca-dev-cleaned.tsv')
dev_tsv = csv.reader(dev_file, delimiter="\t")
total_length = 0
total = 0
for i, line in enumerate(dev_tsv):
    if i == 0: continue
    total += 1
    total_length += len(line[0].split())
    # print(len(line[0].split()))

print(total)
print(total_length)
print(total_length / total)

376
23444
62.351063829787236


In [21]:
# test stats
test_file = open('eca-test-cleaned.tsv')
test_tsv = csv.reader(test_file, delimiter="\t")
total_length = 0
total = 0
for i, line in enumerate(test_tsv):
    if i == 0: continue
    total += 1
    total_length += len(line[0].split())
    # print(len(line[0].split()))

print(total)
print(total_length)
print(total_length / total)

319
19210
60.21943573667711


In [11]:
# train stats
train_file = open('eca-dev-cleaned-exp-TRS-4.tsv')
train_tsv = csv.reader(train_file, delimiter="\t")
total_length = 0
total = 0
for i, line in enumerate(train_tsv):
    if i == 0: continue
    total += 1
    total_length += len(line[-1].strip().split())
    # print(len(line[-1].strip().split()))
    # print((line[-1].strip()))
    # print(len(line[0].split()))

print(total)
print(total_length)
print(total_length / total)

376
12121
32.236702127659576


#### Sample data for choosing prompts

In [2]:
train_df = pd.read_csv('eca-train-cleaned.tsv', delimiter="\t")
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3012 entries, 0 to 3011
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   document       3012 non-null   object
 1   token_label    3012 non-null   object
 2   emotion_label  3012 non-null   object
dtypes: object(3)
memory usage: 70.7+ KB


In [3]:
train_df[train_df['emotion_label'] == 'surprise'].sample(1)

,document,token_label,emotion_label
757,"""Look here "" he began and he was surprised at ...",O O O O O O O O B-EMO O B-CAU I-CAU I-CAU I-CA...,surprise


In [11]:
two_shot_df = pd.read_csv('eca-test-cleaned-exp-TRS-2.tsv', delimiter="\t")
two_shot_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 319 entries, 0 to 318
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   document       319 non-null    object
 1   token_label    319 non-null    object
 2   emotion_label  319 non-null    object
 3   explanation    319 non-null    object
dtypes: object(4)
memory usage: 10.1+ KB


In [12]:
len_lst = []
for exp in two_shot_df['explanation']:
    # print(len(exp))
    len_lst.append(len(str(exp).split()))

print(sum(len_lst) / len(len_lst))

70.20689655172414


### Post-process raw GPT-3 generations to contain only explanations

In [44]:
split = 'dev'
shot = 'TRS-2'
shot_raw_df = pd.read_csv('eca-{}-cleaned-exp-{}-raw.tsv'.format(split, shot), delimiter="\t")
shot_raw_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 376 entries, 0 to 375
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   document       376 non-null    object
 1   token_label    376 non-null    object
 2   emotion_label  376 non-null    object
 3   explanation    376 non-null    object
dtypes: object(4)
memory usage: 11.9+ KB


In [45]:
explanations = []
for sent in shot_raw_df['explanation']:
    if len(sent.split('The cause results in the emotion mentioned because')) != 2:
        explanations.append(sent)
        print(sent)
    else:
        # print(sent.split('The cause results in the emotion mentioned because')[1])
        explanations.append(sent.split('The cause results in the emotion mentioned because')[1])
shot_raw_df['explanations'] = explanations
shot_raw_df = shot_raw_df.drop(['explanation'], axis=1)

 In the text, the cause is that the mother was worried about telling her husband that she had a daughter, not a son. The emotion mentioned is that the husband was delighted when he looked into his daughter's eyes. The cause results in the emotion mentioned because the husband wanted a son, but when he saw his daughter's eyes, he was delighted. The cause results in the emotion mentioned because the husband was happy to have a daughter, even though he originally wanted a son.


In [46]:
shot_raw_df.to_csv('eca-{}-cleaned-exp-{}.tsv'.format(split,shot), sep='\t', index=False)